# Decision Tree
Code from Google YouTube tutorial - ["Machine Learning Recipes \#8. We'll write a Decision Tree Classifier, in pure Python"](https://www.youtube.com/watch?v=LDRbO9a6XPU)  
[Github link](https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb)


In [1]:
!python --version

Python 3.6.9


In [2]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
header = ["color", "diameter", "label"]

In [3]:
def unique_vals(rows, col):
    """Find unique values for a column in a dataset"""
    return set([row[col] for row in rows])

In [4]:
# without set
print([row[0] for row in training_data])

# test
print(unique_vals(training_data, 0))

['Green', 'Yellow', 'Red', 'Red', 'Yellow']
{'Yellow', 'Red', 'Green'}


In [5]:
def class_count(rows):
    """Counts the number of each class in a dataset"""
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [6]:
class_count(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [7]:
def is_numeric(value):
    return isinstance(value, (int, float))

In [8]:
print(is_numeric(7))
print(is_numeric(5.1))

True
True


In [9]:
class Question:
    """A question is used to partition a dataset"""
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __repr__(self):
        condition = '=='
        if is_numeric(self.value):
            condition = '>='
        return 'Is {} {} {}?'.format(header[self.column], condition, self.value)

In [10]:
# test
print(Question(1, 3))
print(Question(0, 'Green'))

Is diameter >= 3?
Is color == Green?


In [11]:
q = Question(0, 'Red')
print(q.match(training_data[0]))
print(q.match(training_data[2]))

False
True


In [12]:
def partition(rows, question):
    """Partions a dataset"""
    true_rows = []
    false_rows = []
    
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [13]:
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
print(true_rows)
print(false_rows)

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


In [14]:
def gini(rows):
    """Calculating the Gini Impurity for a list of rows
    I_G(p) = sum(p_i*(1-p_i)) = sum(p_i-p_i^2)) = sum(p_i) - sum(p_i)^2 = 1-sum(p_i^2)
    """
    counts = class_count(rows)
    impurity = 1
    for label in counts:
        prob_of_label = counts[label] / float(len(rows))
        impurity -= prob_of_label**2
    return impurity

In [15]:
# Test
no_mixing = [['Apple'], ['Apple'], ['Apple']]
print(class_count(no_mixing))
print(gini(no_mixing))

mixed = [['Apple'], ['Apple'], ['Orange'], ['Orange']]
print(class_count(mixed))
print(gini(mixed))

lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
print(gini(lots_of_mixing))

{'Apple': 3}
0.0
{'Apple': 2, 'Orange': 2}
0.5
0.7999999999999998


In [16]:
def info_gain(left, right, current_uncertainty):
    """Information gain
    
    (Uncertainty of the starting node) - (Weighted impurity of two child nodes)
    """
    weight_left = float(len(left)) / float(len(left) + len(right))
    return current_uncertainty - weight_left * gini(left) - (1-weight_left)*gini(right)

In [17]:
current_uncertainty = gini(training_data)
print(current_uncertainty)

true_rows, false_rows = partition(training_data, Question(0, 'Green'))
print(info_gain(true_rows, false_rows, current_uncertainty))

true_rows, false_rows = partition(training_data, Question(0, 'Red'))
print(info_gain(true_rows, false_rows, current_uncertainty))

0.6399999999999999
0.1399999999999999
0.37333333333333324


In [18]:
def find_best_split(rows):
    """Finding the best question to ask by iterating over every feature and 
    calculating the info. gain."""
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 # index starts from 0
    
    for col in range(n_features):
        values = unique_vals(rows, col)
        
        for val in values:
            question = Question(col, val)
            
            # spliting data by the question
            true_rows, false_rows = partition(rows, question)
            
            # skip the trivial question
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            
            # calculating the info. gain
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            # update
            if gain >= best_gain:
                best_gain = gain
                best_question = question
    return best_gain, best_question

In [19]:
best_gain, best_question = find_best_split(training_data)
print(best_gain)
print(best_question)

0.37333333333333324
Is diameter >= 3?


In [20]:
class Leaf:
    """A Leaf node classifies data.
    
    This holds a dictionary of class -> number of items it appears in the rows
    from the training data that reach this leaf.
    """
    def __init__(self, rows):
        self.predictions = class_count(rows)

In [21]:
class Decision_Node:
    """A Decision Node asks a question.
    
    This holds a reference to the question, and to the two child nodes.
    """
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [22]:
def build_tree(rows):
    """Builds the tree.
    
    Rules of recursion: 1) Believe that it works. 2) Starts by checking for the 
    base case (no further information gain). 3) Prepare for giant stack traces.
    """
    gain, question = find_best_split(rows)
    
    # Base case: no further info. gain, we return a leaf.
    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

In [26]:
def print_tree(node, spacing=''):
    if isinstance(node, Leaf):
        print(spacing + 'Predict', node.predictions)
        return
    
    print(spacing + str(node.question))
    print(spacing + '-->True:')
    print_tree(node.true_branch, spacing + ' ')
    
    print(spacing + '-->False:')
    print_tree(node.false_branch, spacing + ' ')

In [27]:
my_tree = build_tree(training_data)

In [28]:
print_tree(my_tree)

Is diameter >= 3?
-->True:
 Is color == Yellow?
 -->True:
  Predict {'Apple': 1, 'Lemon': 1}
 -->False:
  Predict {'Apple': 1}
-->False:
 Predict {'Grape': 2}


In [36]:
def classify(row, node):
    """See the rules of recursion above"""
    if isinstance(node, Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [37]:
# test
classify(training_data[0], my_tree)

{'Apple': 1}

In [39]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + '%'
    return probs
    

In [41]:
print_leaf(classify(training_data[0], my_tree))

{'Apple': '100%'}

In [40]:
print_leaf(classify(training_data[1], my_tree))

{'Apple': '50%', 'Lemon': '50%'}

In [42]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [43]:
for row in testing_data:
    print ("Actual: {}. Predicted: {}".format(
        row[-1], 
        print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
